In [ ]:
import numpy as np
import os
import shutil


from PIL import Image, ImageDraw

In [ ]:
# Step 1: Define the 3D cube vertices and edges (the cube will be still)
cube_vertices = np.array([
    [1, 1, -1], [-1, 1, -1], [-1, -1, -1], [1, -1, -1],
    [1, 1, 1], [-1, 1, 1], [-1, -1, 1], [1, -1, 1]
])

cube_edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),
    (4, 5), (5, 6), (6, 7), (7, 4),
    (0, 4), (1, 5), (2, 6), (3, 7)
]

In [ ]:
def normalize(v):
  """
  Normalize a given vector.
  """
  norm = np.linalg.norm(v)
  if norm == 0:
    return v
  return v / norm

In [ ]:
def look_at(from_pos, to_pos, up):
    forward = normalize(from_pos - to_pos)
    right = normalize(np.cross(up, forward))
    up = np.cross(forward, right)
    view_matrix = np.array([
        [right[0], right[1], right[2], -np.dot(right, from_pos)],
        [up[0], up[1], up[2], -np.dot(up, from_pos)],
        [forward[0], forward[1], forward[2], -np.dot(forward, from_pos)],
        [0, 0, 0, 1]
    ])
    return view_matrix

In [ ]:
def project_vertex(vertex, view_matrix, projection_matrix):
    vertex_homogeneous = np.append(vertex, 1)
    transformed_vertex = view_matrix.dot(vertex_homogeneous)
    projected_vertex = projection_matrix.dot(transformed_vertex)
    return projected_vertex[:2] / projected_vertex[3]

In [ ]:
# Initial camera setup
initial_camera_pos = np.array([7.0, 1.0, 1.0])
target_pos = np.array([0.0, 0.0, 0.0])
up_vector = np.array([0.0, 1.0, 0.0])

In [ ]:
# Projection matrix setup (assuming perspective projection)
fov = 60  # Field of view in degrees
aspect_ratio = 1.0  # Aspect ratio
near = 1.0  # Near clipping plane
far = 100.0  # Far clipping plane
fov_rad = np.radians(fov)
f = 1 / np.tan(fov_rad / 2)
projection_matrix = np.array([
    [f / aspect_ratio, 0, 0, 0],
    [0, f, 0, 0],
    [0, 0, (far + near) / (near - far), (2 * far * near) / (near - far)],
    [0, 0, -1, 0]
])

In [ ]:
# Rendering parameters
radius_range = 5
num_steps = 100

In [ ]:
# Ensure the output directory exists
render_dir = "renders_pillow"
if not os.path.exists(render_dir):
    os.makedirs(render_dir)

In [ ]:
# Image rendering loop
r = radius_range
for step in range(num_steps):
    alpha = step * 2 * np.pi / num_steps
    camera_pos = np.array([
        target_pos[0] + r * np.cos(alpha),
        target_pos[1] + r * np.sin(alpha),
        initial_camera_pos[2]
    ])
    view_matrix = look_at(camera_pos, target_pos, up_vector)
    image_size = 500
    image = Image.new("RGB", (image_size, image_size), "white")
    draw = ImageDraw.Draw(image)

    # Project and draw each edge of the cube
    for edge in cube_edges:
        v0, v1 = cube_vertices[edge[0]], cube_vertices[edge[1]]
        p0 = project_vertex(v0, view_matrix, projection_matrix)
        p1 = project_vertex(v1, view_matrix, projection_matrix)
        p0 = (image_size / 2 * (p0 + 1)).astype(int)
        p1 = (image_size / 2 * (p1 + 1)).astype(int)
        draw.line([tuple(p0), tuple(p1)], fill="black")

    # Save the image
    filename = f'{render_dir}/frame_{r}_{step}.png'
    image.save(filename)

print(f'Rendering completed. Images saved in {render_dir} directory.')

In [ ]:
# Ensure the output directory exists in the current working directory
output_dir = "/cube_images_fixed"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the output directory
output_dir = "/cube_images_fixed"
zip_filename = "/cube_images_fixed.zip"

# Copy images to the output directory
for file in os.listdir(render_dir):
    shutil.copy(os.path.join(render_dir, file), output_dir)

# Create a zip file
shutil.make_archive("/cube_images_fixed", 'zip', output_dir)

print(f'Images are saved and zipped in {zip_filename}')
